In [1]:
import numpy as np
from itertools import cycle
import os
import dask.array as da
import dask.bag as db
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xesmf as xe
import subprocess
from scipy.ndimage import binary_fill_holes
from importlib import reload
os.chdir("/home/149/ab8992/cosima_regional/mom6-regional-scripts")
import regional_library as ml
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43849,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:33605,Total threads: 4
Dashboard: /proxy/44803/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:37961,


# Test run for Glory's input, Gebco bathy and JRA forcing

In [2]:
reload(ml)

## Choose your coordinates and the name of your experiment
yextent = [-48,-38.95]
xextent = [-217 + 360, -210 + 360]
expt_name = "yearlongTassie"

pwd = "$KQ%QqFxjSSbE2"
usr = "abarnes"


daterange = ["2015-01-01 00:00:00", "2015-12-31 00:00:00"]

## Place where all your input files go
inputdir = f"/scratch/v45/ab8992/mom6/regional_configs/{expt_name}/"

## Directory where you'll run the experiment from
rundir = f"/home/149/ab8992/mom6_rundirs/{expt_name}/"

## Directory where raw downloads go before processing
tmpdir = f"/scratch/v45/ab8992/reanalysis_tmp/reanalysis-small/"

## Directory where fre tools are stored
toolpath = "/home/157/ahg157/repos/mom5/src/tools/" ## Compiled tools needed for construction of mask tables


## Preamble: Use the moturequests function to pull down the data we want from Glorys dataset


In [5]:

file = open(f"{tmpdir}/get_oceanfiles.sh","w")
file.write(
        ml.motu_requests(xextent, yextent, daterange, tmpdir, usr, pwd,["north","south","east","west"])
)
file.close()


### First make an experiment object
This keeps track of your domain basics, as well as generating the hgrid, vgrid and setting up the folder structures. 

Sanity check: After running you a sensible hgrid and vgrind should appear in your input directory.

In [3]:
reload(ml)
expt = ml.experiment(
    xextent,
    yextent,
    daterange,
    0.05,  ## Resolution
    75,      ## #zlayers
    20,       ## dz ratio
    4000,    ## Max depth of ocean
    rundir,
    inputdir,
    toolpath
)

FRE TOOLS: Make hgrid 

 CompletedProcess(args=['/home/157/ahg157/repos/mom5/src/tools/make_hgrid/make_hgrid', '--grid_type', 'from_file', '--my_grid_file', 'grid.nc'], returncode=0)
NOTE from make_solo_mosaic: there are 0 contacts (align-contact)
congradulation: You have successfully run make_solo_mosaic
FRE TOOLS: Make solo mosaic


CompletedProcess(args=['/home/157/ahg157/repos/mom5/src/tools/make_solo_mosaic/make_solo_mosaic', '--num_tiles', '1', '--dir', '.', '--mosaic_name', 'ocean_mosaic', '--tile_file', 'hgrid.nc'], returncode=0)


ln: failed to create symbolic link '/home/149/ab8992/mom6_rundirs/yearlongTassie//inputdir/yearlongTassie': File exists
ln: failed to create symbolic link '/scratch/v45/ab8992/mom6/regional_configs/yearlongTassie//rundir/yearlongTassie': File exists


## Next, handle the ocean forcing.

This cuts out and interpolates the initial condition as well as all boundaries (unless you don't pass it boundaries).

If you haven't used the script that pulls down the Glorys data, be sure to cut out your boundaries and name them like "north_segment_unprocessed", and your IC like "ic_segment_unprocessed"

In [13]:
## Patch up encoding for time. This is bandaid solution for now
i = 1
if "test" not in os.listdir(f"/home/149/ab8992/mom6_rundirs/{expt_name}/inputdir/forcing"):
    os.mkdir(f"/home/149/ab8992/mom6_rundirs/{expt_name}/inputdir/forcing/test")

for i in range(1,5):
    old = xr.open_dataset(f"/home/149/ab8992/mom6_rundirs/{expt_name}/inputdir/forcing/forcing_obc_segment_00{i}.nc",decode_times=False)
    # old = old.assign_coords({"time":np.arange(41975,41975 + 365,dtype=float)})
    old = old.assign_coords({"time":np.arange(0,365,dtype=float)})
    old.time.attrs["units"] = "days since 2015-01-01"
    old.time.attrs["calendar"] = "noleap"
    old.to_netcdf(f"/home/149/ab8992/mom6_rundirs/{expt_name}/inputdir/forcing/test/forcing_obc_segment_00{i}.nc")

subprocess.run(f"mv forcing/test/* forcing/" , shell=True,cwd = inputdir)

CompletedProcess(args='mv forcing/test/* forcing/', returncode=0)

In [11]:
a = expt.ocean_forcing(
    tmpdir,  ## Path to ocean foring files
    {"time":"time","y":"latitude","x":"longitude",
     "zl":"depth","eta":"zos","u":"uo","v":"vo",
     "tracers":{"salt":"so","temp":"thetao"}},
    boundaries = ["south","north","west","east"]
    )

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/core/dataarray.py:854: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Wi

In [10]:
expt.hgrid

<xarray.Dataset>
Dimensions:   (nyp: 361, nxp: 281, nx: 280, ny: 360)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    tile      |S255 ...
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    arcx      |S255 ...
Attributes:
    grid_version:  0.2
    code_version:  $Name: tikal $
    history:       /home/157/ahg157/repos/mom5/src/tools/make_hgrid/make_hgri...

In [5]:
expt.bathymetry(
    '/g/data/ik11/inputs/GEBCO_2022/GEBCO_2022.nc',
    {"xh":"lon","yh":"lat","elevation":"elevation"},
    )


==>NOTE from get_boundary_type: x_boundary_type is solid_walls

==>NOTE from get_boundary_type: y_boundary_type is solid_walls
FRE TOOLS: make topog parallel

 CompletedProcess(args=['/g/data/v45/jr5971/FRE-NCtools/build3_up_MAXXGRID/tools/make_topog/make_topog_parallel', '--mosaic', 'ocean_mosaic.nc', '--topog_type', 'realistic', '--topog_file', 'bathy_original.nc', '--topog_field', 'elevation', '--scale_factor', '-1', '--output', 'topog_raw.nc'], returncode=0)
NOTE from make_solo_mosaic: there are 0 contacts (align-contact)
congradulation: You have successfully run make_solo_mosaic
MAKE SOLO MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_solo_mosaic/make_solo_mosaic --num_tiles 1 --dir . --mosaic_name ocean_mosaic --tile_file hgrid.nc', returncode=0)


cp: './ocean_mosaic.nc' and 'ocean_mosaic.nc' are the same file
cp: './hgrid.nc' and 'hgrid.nc' are the same file


cp ./hgrid.nc hgrid.nc 

NOTE from make_coupler_mosaic: the ocean land/sea mask will be determined by field depth from file topog.nc
mosaic_file is grid_spec.nc

***** Congratulation! You have successfully run make_quick_mosaic
QUICK MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_quick_mosaic/make_quick_mosaic --input_mosaic ocean_mosaic.nc --mosaic_name grid_spec --ocean_topog topog.nc', returncode=0)

 ===>NOTE from check_mask: when layout is specified, min_pe and max_pe is set to layout(1)*layout(2)=100

 ===>NOTE from check_mask: Below is the list of command line arguments.

grid_file = ocean_mosaic.nc
topog_file = topog.nc
min_pe = 100
max_pe = 100
layout = 10, 10
halo = 4
sea_level = 0
show_valid_only is not set
nobc = 0

 ===>NOTE from check_mask: End of command line arguments.

 ===>NOTE from check_mask: the grid file is version 2 (mosaic grid) grid which contains field gridfiles

==>NOTE from get_boundary_type: x_boundary_type is solid_walls

==>NOTE

In [ ]:
expt.processor_mask((10,10))

: 

## Things to handle:
* Forcing files that have negative longitudes. Should ensure that xextent always means the same thing
* Have all of my compiled fre tools together in the same folder

## Below is stuff copied from the old notebook that's not in pipeline. 
* Runoff
* Make mask
* copy default run directory

In [11]:
from regional_model_scripts import regrid_runoff
runoff_path = "/g/data/ik11/inputs/JRA-55/RYF/v1-3/RYF.runoff_all.1990_1991.nc"

regrid_runoff(inputdir + "ocean_mask.nc",
    inputdir + "hgrid.nc",
    runoff_path,
    inputdir + "runoff_regrid.nc",
    np.array(xextent) - np.array([180,180]),
    yextent)

## Modify the default input directory to make a (hopefully) runnable configuration out of the box

In [12]:
subprocess.run(f"cp default_rundir/* {rundir}",shell = True)
subprocess.run(f"ln -s {inputdir} {rundir}/inputdir",shell=True)

hgrid = xr.open_dataset(f"{inputdir}/hgrid.nc")

## Get mask table information
ncpus = 100
mask_table = None
for i in os.listdir(f"{inputdir}"):
    if "mask_table" in i:
        mask_table = i
        a = mask_table.split(".")[1]
        b = mask_table.split(".")[2].split("x")
        ncpus = int(b[0]) * int(b[1]) - int(a)


## Modify MOM_input
inputfile = open(f"{rundir}/MOM_input",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "MASKTABLE" in lines[i]:
        if mask_table != None:
            lines[i] = f'MASKTABLE = "{mask_table}"\n'
        else:
            lines[i] = "# MASKTABLE = no mask table"
    if "NIGLOBAL" in lines[i]: 
        # lines[i] = f"NIGLOBAL = {str(x_indices_centre[1] - x_indices_centre[0])}\n"
        lines[i] = f"NIGLOBAL = {hgrid.nx.shape[0]//2}\n"

        
    if "NJGLOBAL" in lines[i]:
        # lines[i] = f"NJGLOBAL = {str(y_indices_centre[1] - y_indices_centre[0])}\n"
        lines[i] = f"NJGLOBAL = {hgrid.ny.shape[0]//2}\n"

        
inputfile = open(f"{rundir}/MOM_input",'w')

inputfile.writelines(lines)
inputfile.close()

## Modify SIS_input
inputfile = open(f"{rundir}/SIS_input",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "MASKTABLE" in lines[i]:
        lines[i] = f'MASKTABLE = "{mask_table}"\n'
    if "NIGLOBAL" in lines[i]:
        # lines[i] = f"NIGLOBAL = {str(x_indices_centre[1] - x_indices_centre[0])}\n"
        lines[i] = f"NIGLOBAL = {hgrid.nx.shape[0]//2}\n"
        
    if "NJGLOBAL" in lines[i]:
        # lines[i] = f"NJGLOBAL = {str(y_indices_centre[1] - y_indices_centre[0])}\n"
        lines[i] = f"NJGLOBAL = {hgrid.ny.shape[0]//2}\n"
        
inputfile = open(f"{rundir}/SIS_input",'w')
inputfile.writelines(lines)
inputfile.close()

## Modify config.yaml 
inputfile = open(f"{rundir}/config.yaml",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "ncpus" in lines[i]:
        lines[i] = f'ncpus: {str(ncpus)}\n'
    if "jobname" in lines[i]:
        lines[i] = f"jobname: mom6_{expt_name}\n"
        
    if "input:" in lines[i]:
        lines[i + 1] = f"    - {inputdir}\n"
        
inputfile = open(f"{rundir}/config.yaml",'w')
inputfile.writelines(lines)
inputfile.close()


ln: failed to create symbolic link '/home/149/ab8992/mom6_rundirs/yearlongTassie30th//inputdir/yearlongTassie30th': File exists


# Test number 2:
## Same as above, but with ACCESS OM2 forcing instead. 

In [7]:
reload(ml)

## Choose your coordinates and the name of your experiment
yextent = [-48,-38.95]
xextent = [-217 + 360, -210 + 360]
expt_name = "om2-input"

pwd = "$KQ%QqFxjSSbE2"
usr = "abarnes"


daterange = ["1901-01-01 00:00:00", "1901-12-31 00:00:00"]

## Place where all your input files go
inputdir = f"/scratch/v45/ab8992/mom6/regional_configs/{expt_name}/"

## Directory where you'll run the experiment from
rundir = f"/home/149/ab8992/mom6_rundirs/{expt_name}/"

## Directory where raw downloads go before processing
tmpdir = f"/scratch/v45/ab8992/mom6/regional_configs/{expt_name}/rawboundaries"

## Directory where fre tools are stored
toolpath = "/home/157/ahg157/repos/mom5/src/tools/" ## Compiled tools needed for construction of mask tables

expt = ml.experiment(
    xextent,
    yextent,
    0.0333,  ## Resolution
    75,      ## #zlayers
    20,       ## dz ratio
    4000,    ## Max depth of ocean
    rundir,
    inputdir,
    toolpath
)

FRE TOOLS: Make hgrid 

 CompletedProcess(args=['/home/157/ahg157/repos/mom5/src/tools/make_hgrid/make_hgrid', '--grid_type', 'from_file', '--my_grid_file', 'grid.nc'], returncode=0)
NOTE from make_solo_mosaic: there are 0 contacts (align-contact)
congradulation: You have successfully run make_solo_mosaic
FRE TOOLS: Make solo mosaic


CompletedProcess(args=['/home/157/ahg157/repos/mom5/src/tools/make_solo_mosaic/make_solo_mosaic', '--num_tiles', '1', '--dir', '.', '--mosaic_name', 'ocean_mosaic', '--tile_file', 'hgrid.nc'], returncode=0)


ln: failed to create symbolic link '/home/149/ab8992/mom6_rundirs/om2-input//inputdir/om2-input': File exists
ln: failed to create symbolic link '/scratch/v45/ab8992/mom6/regional_configs/om2-input//rundir/om2-input': File exists


**First step is to cut out data into the five slices we need. This time need to manually cut up the OM2 data into the 5 netcdfs ourself (NSEW + Initial)**

Please forgive the repeated / hardcoded lines this could be tidied up later

In [8]:
om2_input = xr.concat(
    [xr.open_mfdataset(f"/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output{i}/ocean/ocean_daily*",decode_times = False,parallel=True,chunks='auto') for i in range(1077,1082)]
)

reload(ml)
## Cut out initial condition and save
ic = om2_input[["u","v","salt","temp","eta_t"]].sel(    
    yu_ocean = slice(yextent[0] - 0.2,yextent[1] + 0.2),
    yt_ocean = slice(yextent[0] - 0.2,yextent[1] + 0.2)
).isel(time = 0)

## Nicer Slicer handles seams in longitude and different grids. Ensures that the output matches our 'xextend'
ic = ml.nicer_slicer(ic,[xextent[0],xextent[1]],["xu_ocean","xt_ocean"])
ic.to_netcdf(tmpdir + "/ic_unprocessed")

eastwest = om2_input[["u","v","salt","temp","eta_t"]].sel(    
    yu_ocean = slice(yextent[0] - 0.2,yextent[1] + 0.2),
    yt_ocean = slice(yextent[0] - 0.2,yextent[1] + 0.2)
)
ml.nicer_slicer(eastwest,[xextent[1],xextent[1]],["xu_ocean","xt_ocean"]).to_netcdf(tmpdir + "/east_unprocessed")
ml.nicer_slicer(eastwest,[xextent[0],xextent[0]],["xu_ocean","xt_ocean"]).to_netcdf(tmpdir + "/west_unprocessed")

northsouth = ml.nicer_slicer(om2_input[["u","v","salt","temp","eta_t"]],[xextent[0],xextent[1]],["xu_ocean","xt_ocean"])
northsouth.sel(
    yu_ocean = slice(yextent[1] - 0.2,yextent[1] + 0.2),
    yt_ocean = slice(yextent[1] - 0.2,yextent[1] + 0.2)
).to_netcdf(tmpdir + "/north_unprocessed")
northsouth.sel(
    yu_ocean = slice(yextent[0] - 0.2,yextent[0] + 0.2),
    yt_ocean = slice(yextent[0] - 0.2,yextent[0] + 0.2)
).to_netcdf(tmpdir + "/south_unprocessed")

In [10]:
om2_input

<xarray.Dataset>
Dimensions:                (xt_ocean: 3600, yt_ocean: 2700, st_ocean: 75,
                            st_edges_ocean: 76, time: 91, nv: 2,
                            xu_ocean: 3600, yu_ocean: 2700, sw_ocean: 75,
                            sw_edges_ocean: 76)
Coordinates:
  * xt_ocean               (xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * yt_ocean               (yt_ocean) float64 -81.11 -81.07 ... 89.94 89.98
  * st_ocean               (st_ocean) float64 0.5413 1.681 ... 5.709e+03
  * st_edges_ocean         (st_edges_ocean) float64 0.0 1.083 ... 5.809e+03
  * time                   (time) float64 9.864e+04 9.864e+04 ... 9.873e+04
  * nv                     (nv) float64 1.0 2.0
  * xu_ocean               (xu_ocean) float64 -279.9 -279.8 -279.7 ... 79.9 80.0
  * yu_ocean               (yu_ocean) float64 -81.09 -81.05 -81.0 ... 89.96 90.0
  * sw_ocean               (sw_ocean) float64 1.083 2.279 ... 5.61e+03 5.809e+03
  * sw_edges_ocean         (sw_edges_ocean) float64 0.5128 1.652 ... 5.809e+03
Data variables: (12/26)
    dzt                    (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    average_T1             (time) float64 dask.array<chunksize=(91,), meta=np.ndarray>
    average_T2             (time) float64 dask.array<chunksize=(91,), meta=np.ndarray>
    average_DT             (time) float64 dask.array<chunksize=(91,), meta=np.ndarray>
    time_bounds            (time, nv) float64 dask.array<chunksize=(91, 2), meta=np.ndarray>
    eta_t                  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(30, 915, 1220), meta=np.ndarray>
    ...                     ...
    temp                   (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    u                      (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    uhrho_et               (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    v                      (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    vhrho_nt               (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
    wt                     (time, sw_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(13, 11, 418, 558), meta=np.ndarray>
Attributes:
    filename:   ocean_daily_3d_dzt.nc
    title:      ACCESS-OM2-01
    grid_type:  mosaic
    grid_tile:  1

In [9]:
a = expt.ocean_forcing(
    tmpdir,  ## Path to ocean foring files
    {"time":"time","yh":"yt_ocean","xh":"xt_ocean",
     "xq":"xu_ocean","yq":"yu_ocean",
     "zl":"st_ocean","eta":"eta_t","u":"u","v":"v",
     "tracers":{"salt":"salt","temp":"temp"}},
    boundaries = ["south","north","west","east"],
    gridtype="B"
    )

/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xarray/core/dataarray.py:854: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.10/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Wi

In [26]:
expt.bathymetry(
    '/g/data/ik11/inputs/GEBCO_2022/GEBCO_2022.nc',
    {"xh":"lon","yh":"lat","elevation":"elevation"},
    )


==>NOTE from get_boundary_type: x_boundary_type is solid_walls

==>NOTE from get_boundary_type: y_boundary_type is solid_walls
FRE TOOLS: make topog parallel

 CompletedProcess(args=['/g/data/v45/jr5971/FRE-NCtools/build3_up_MAXXGRID/tools/make_topog/make_topog_parallel', '--mosaic', 'ocean_mosaic.nc', '--topog_type', 'realistic', '--topog_file', 'bathy_original.nc', '--topog_field', 'elevation', '--scale_factor', '-1', '--output', 'topog_raw.nc'], returncode=0)
NOTE from make_solo_mosaic: there are 0 contacts (align-contact)
congradulation: You have successfully run make_solo_mosaic
MAKE SOLO MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_solo_mosaic/make_solo_mosaic --num_tiles 1 --dir . --mosaic_name ocean_mosaic --tile_file hgrid.nc', returncode=0)


cp: './ocean_mosaic.nc' and 'ocean_mosaic.nc' are the same file
cp: './hgrid.nc' and 'hgrid.nc' are the same file


cp ./hgrid.nc hgrid.nc 

NOTE from make_coupler_mosaic: the ocean land/sea mask will be determined by field depth from file topog.nc
mosaic_file is grid_spec.nc

***** Congratulation! You have successfully run make_quick_mosaic
QUICK MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_quick_mosaic/make_quick_mosaic --input_mosaic ocean_mosaic.nc --mosaic_name grid_spec --ocean_topog topog.nc', returncode=0)

 ===>NOTE from check_mask: when layout is specified, min_pe and max_pe is set to layout(1)*layout(2)=100

 ===>NOTE from check_mask: Below is the list of command line arguments.

grid_file = ocean_mosaic.nc
topog_file = topog.nc
min_pe = 100
max_pe = 100
layout = 10, 10
halo = 4
sea_level = 0
show_valid_only is not set
nobc = 0

 ===>NOTE from check_mask: End of command line arguments.

 ===>NOTE from check_mask: the grid file is version 2 (mosaic grid) grid which contains field gridfiles

==>NOTE from get_boundary_type: x_boundary_type is solid_walls

==>NOTE

## Below is stuff copied from the old notebook that's not in pipeline. 
* Runoff
* Make mask
* copy default run directory

In [27]:
from regional_model_scripts import regrid_runoff
runoff_path = "/g/data/ik11/inputs/JRA-55/RYF/v1-3/RYF.runoff_all.1990_1991.nc"

regrid_runoff(inputdir + "ocean_mask.nc",
    inputdir + "hgrid.nc",
    runoff_path,
    inputdir + "runoff_regrid.nc",
    np.array(xextent) - np.array([180,180]),
    yextent)

In [28]:
subprocess.run(f"cp default_rundir/* {rundir}",shell = True)
subprocess.run(f"ln -s {inputdir} {rundir}/inputdir",shell=True)

hgrid = xr.open_dataset(f"{inputdir}/hgrid.nc")

## Get mask table information
ncpus = 100




































































































-¬
mask_table = None
for i in os.listdir(f"{inputdir}"):
    if "mask_table" in i:
        mask_table = i
        a = mask_table.split(".")[1]
        b = mask_table.split(".")[2].split("x")
        ncpus = int(b[0]) * int(b[1]) - int(a)


## Modify MOM_input
inputfile = open(f"{rundir}/MOM_input",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "MASKTABLE" in lines[i]:
        if mask_table != None:
            lines[i] = f'MASKTABLE = "{mask_table}"\n'
        else:
            lines[i] = "# MASKTABLE = no mask table"
    if "NIGLOBAL" in lines[i]: 
        # lines[i] = f"NIGLOBAL = {str(x_indices_centre[1] - x_indices_centre[0])}\n"
        lines[i] = f"NIGLOBAL = {hgrid.nx.shape[0]//2}\n"

        
    if "NJGLOBAL" in lines[i]:
        # lines[i] = f"NJGLOBAL = {str(y_indices_centre[1] - y_indices_centre[0])}\n"
        lines[i] = f"NJGLOBAL = {hgrid.ny.shape[0]//2}\n"

        
inputfile = open(f"{rundir}/MOM_input",'w')

inputfile.writelines(lines)
inputfile.close()

## Modify SIS_input
inputfile = open(f"{rundir}/SIS_input",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "MASKTABLE" in lines[i]:
        lines[i] = f'MASKTABLE = "{mask_table}"\n'
    if "NIGLOBAL" in lines[i]:
        # lines[i] = f"NIGLOBAL = {str(x_indices_centre[1] - x_indices_centre[0])}\n"
        lines[i] = f"NIGLOBAL = {hgrid.nx.shape[0]//2}\n"
        
    if "NJGLOBAL" in lines[i]:
        # lines[i] = f"NJGLOBAL = {str(y_indices_centre[1] - y_indices_centre[0])}\n"
        lines[i] = f"NJGLOBAL = {hgrid.ny.shape[0]//2}\n"
        
inputfile = open(f"{rundir}/SIS_input",'w')
inputfile.writelines(lines)
inputfile.close()

## Modify config.yaml 
inputfile = open(f"{rundir}/config.yaml",'r')
lines = inputfile.readlines()
inputfile.close()
for i in range(len(lines)):
    if "ncpus" in lines[i]:
        lines[i] = f'ncpus: {str(ncpus)}\n'
    if "jobname" in lines[i]:
        lines[i] = f"jobname: mom6_{expt_name}\n"
        
    if "input:" in lines[i]:
        lines[i + 1] = f"    - {inputdir}\n"
        
inputfile = open(f"{rundir}/config.yaml",'w')
inputfile.writelines(lines)
inputfile.close()

ln: failed to create symbolic link '/home/149/ab8992/mom6_rundirs/om2-input//inputdir/om2-input': File exists
